Feb 28

It has turned out that the validation of the current
ower-fb-3 dataset are not useful as all the ground truth
classes are false for the first 300 entities.

The script for building the OWER dataset needs to be
examined and corrected. Estimated, the most frequent
classes should be true at least for every 100th entity.